## 1.2 数据清洗

### 导入所需库

In [1]:
import pandas as pd
import numpy as np

### 1.2.1 缺失值

#### 查找缺失值

In [3]:
# 示例数据
df = pd.DataFrame({
    'A': [1, np.nan, 3],
    'B': [4, 5, np.nan]
})
print(df.head())

     A    B
0  1.0  4.0
1  NaN  5.0
2  3.0  NaN


In [8]:
# 查看每列的缺失情况
for i in df.columns:
    print(f'列{i}存在缺失值{ df[i].isnull().sum()}个')

列A存在缺失值1个
列B存在缺失值1个


#### 固定值填充

In [9]:
# 使用固定值0进行填充
df_filled = df.fillna(0)
print(df_filled.head())

     A    B
0  1.0  4.0
1  0.0  5.0
2  3.0  0.0


#### 模型填充

In [10]:
from sklearn.impute import KNNImputer

# 创建KNN填充器
imputer = KNNImputer(n_neighbors=2)

# 使用KNN填充缺失值
df_filled_knn = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

print(df_filled_knn.head())

     A    B
0  1.0  4.0
1  2.0  5.0
2  3.0  4.5


### 1.2.2 异常值

#### 基于统计值的异常值检测

In [26]:
# 示例数据
data = {'value': [10, 12, 12, 13, 12, 11, 500]}
df = pd.DataFrame(data)
print(df)

   value
0     10
1     12
2     12
3     13
4     12
5     11
6    500


In [27]:
# 计算均值和标准差
mean = df['value'].mean()
std = df['value'].std()

# 定义异常值的阈值
threshold = 2

# 检测异常值
outliers = df[(df['value'] < mean - threshold * std) | (df['value'] > mean + threshold * std)]
print(outliers)

   value
6    500


#### 基于DBSCAN的异常值检测

In [19]:
from sklearn.cluster import DBSCAN
import numpy as np

# 假设样例数据集
data = np.array([[10], [12], [12], [13], [12], [11], [500]])

# 应用DBSCAN
dbscan = DBSCAN(eps=3, min_samples=2).fit(data)

# 找出噪声点（异常值）
outlier_indexes = np.where(dbscan.labels_ == -1)[0]
outliers = data[outlier_indexes]
print(outliers)

[[500]]


#### 异常值删除

In [25]:
# 对于示例数据，继续使用上面的统计方法检测到的异常值
df_cleaned = df.drop(outliers.index)
print(df_cleaned)

   value
0     10
1     12
2     12
3     13
4     12
5     11


#### 异常值转换

In [28]:
# 使用均值替换异常值
df['value'][outliers.index] = mean
print(df)

       value
0  10.000000
1  12.000000
2  12.000000
3  13.000000
4  12.000000
5  11.000000
6  81.428571


### 1.2.3 重复值

#### 重复值删除

In [31]:
# 创建样例数据
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'Alice', 'Bob'],
    'Age': [25, 30, 35, 25, 30],
    'Salary': [50000, 60000, 70000, 50000, 60000]
}

df = pd.DataFrame(data)
print(df)

      Name  Age  Salary
0    Alice   25   50000
1      Bob   30   60000
2  Charlie   35   70000
3    Alice   25   50000
4      Bob   30   60000


In [32]:
df_cleaned = df.drop_duplicates()
print(df_cleaned)

      Name  Age  Salary
0    Alice   25   50000
1      Bob   30   60000
2  Charlie   35   70000


#### 重复值聚合

In [33]:
# 创建样例数据
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'Alice', 'Bob'],
    'Age': [25, 30, 35, 25, 30],
    'Salary': [50000, 60000, 70000, 50000, 60000]
}

df = pd.DataFrame(data)
print(df)

      Name  Age  Salary
0    Alice   25   50000
1      Bob   30   60000
2  Charlie   35   70000
3    Alice   25   50000
4      Bob   30   60000


In [35]:
# 假设这里的重复值表示不同时间的薪资记录，我们可以对薪资进行聚合
df_aggregated = df.groupby(['Name', 'Age'], as_index=False).agg({'Salary': 'sum'})
print(df_aggregated)

      Name  Age  Salary
0    Alice   25  100000
1      Bob   30  120000
2  Charlie   35   70000
